In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [3]:
# Add a transform that creates a tensor from the PIL image by adding transform:
transform = transforms.Compose([
    # you can add other transformations in this list
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# download CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# Create Dataloader
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testLoader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# model 
class SimpleNet(nn.Module):
  def __init__(self, input_size, num_class):
    super(SimpleNet, self).__init__()
    self.input = nn.Linear(input_size, 32)
    self.relu = nn.ReLU()
    self.hidden_layer = nn.Linear(32, num_class)
  
  def forward(self, x):
    out = self.input(x)
    out = self.relu(out)
    out = self.hidden_layer(out)

    return out

In [5]:
# Parameters
classes = trainset.classes
class_num = len(classes)
input_size = 32*32*3

net = SimpleNet(input_size=input_size, num_class=class_num).to(device)
print(net)

SimpleNet(
  (input): Linear(in_features=3072, out_features=32, bias=True)
  (relu): ReLU()
  (hidden_layer): Linear(in_features=32, out_features=10, bias=True)
)


In [7]:
criterion = nn.CrossEntropyLoss()
lr = 0.001
epochs = 3
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

In [8]:
# Train
for epoch in range(epochs):
    running_loss = 0.0
    for times, (images, labels) in enumerate(trainLoader):
        # images are 64 (batch_size) *  32*32*3 tensor.
        images = Variable(images.view(-1, 32*32*3)) # -1 means 8 here
        labels = Variable(labels)
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (times+1) % 500 == 0:
            print('Epoch: {}, Batch: {}, Loss: {}'.format(epoch+1, times+1, loss.data))

print('Finished Training')

Epoch: 1, Batch: 500, Loss: 1.7056517601013184
Epoch: 1, Batch: 1000, Loss: 2.19059419631958
Epoch: 1, Batch: 1500, Loss: 1.2345741987228394
Epoch: 1, Batch: 2000, Loss: 1.6936646699905396
Epoch: 1, Batch: 2500, Loss: 2.197434425354004
Epoch: 1, Batch: 3000, Loss: 1.9401494264602661
Epoch: 1, Batch: 3500, Loss: 1.563801884651184
Epoch: 1, Batch: 4000, Loss: 2.572697639465332
Epoch: 1, Batch: 4500, Loss: 1.5126606225967407
Epoch: 1, Batch: 5000, Loss: 1.3178306818008423
Epoch: 1, Batch: 5500, Loss: 1.880400538444519
Epoch: 1, Batch: 6000, Loss: 1.5783478021621704
Epoch: 2, Batch: 500, Loss: 1.8320305347442627
Epoch: 2, Batch: 1000, Loss: 1.4147154092788696
Epoch: 2, Batch: 1500, Loss: 0.982157289981842
Epoch: 2, Batch: 2000, Loss: 1.2556840181350708
Epoch: 2, Batch: 2500, Loss: 1.1594175100326538
Epoch: 2, Batch: 3000, Loss: 1.5146551132202148
Epoch: 2, Batch: 3500, Loss: 1.5738147497177124
Epoch: 2, Batch: 4000, Loss: 1.5828971862792969
Epoch: 2, Batch: 4500, Loss: 1.729058861732483
Ep

In [9]:
# calculate accuracy
correct_num = 0
total_num = 0

for images, labels in testLoader:
    images = Variable(images.view(-1, 32*32*3))
    labels = Variable(labels)

    images, labels = images.to(device), labels.to(device)
    
    # output is 100x10 matrix;
    output = net(images)
    
    # 1 means max by column;
    # _ is -> value; predicted -> label(index).
    _, predicted = torch.max(output.data, 1) 
    
    total_num += labels.size(0) # len;
    correct_num += (predicted == labels).sum()
    
print('accuracy: {} %'.format(float(correct_num)/ float(total_num) * 100))

accuracy: 46.82 %
